# Pipeline for runing batch hidef community detection with input 

## Use the input data to evaluate the coverage of the output MuSIC map 

Goals:
* run the community detection of input 1.) DenseNet embedding 2.) Node2Vec embedding  3.)Original Bioplex network 
    * with the same parameter combination and with bad hierarchies removed 

* Add HPA annotated compartments/locations for geneset enrichment analysis 


In [69]:
import pandas as pd
import numpy as np
import pickle
import sys
from cellmaps_utils.music_utils import *

from tqdm import tqdm
import os
from statsmodels.stats.multitest import multipletests
from scipy.stats import hypergeom, mannwhitneyu, ks_2samp

In [189]:
def run_hypergeo_enrichr(ont_ts, hierarchygenes, ref_file, fdr_thre=0.01, ji_thre = 0, minCompSize=4):  
    '''
    ont_ts: the result hierarchy from the community detection 
    hierarchy genes: total number of genes in the root of the hierarchy 
    ref_file: the reference cellular component/protein complexes table 
    fdr_thre: the fdr cutoff to be collected in the enriched terms (default = 0.01)
    ji_thre: the threshold for Jaccard Idex (default = 0, will set the threshold in the organization step)
    minCompSize: the smallest component size to be considered for the enrichment (default = 4)
    '''
    
    M = len(hierarchygenes)
    
    track = 0
    ref_df = pd.DataFrame(index=ont_ts.index, columns=ref_file.index, dtype=float)
    ji_df = pd.DataFrame(index=ont_ts.index, columns=ref_file.index, dtype=float)
    genecount_df = pd.DataFrame(index=ont_ts.index, columns=ref_file.index, dtype=float)
    for ont_comp, ont_row in tqdm(ont_ts.iterrows(), total=ont_ts.shape[0]):
        track += 1
        ont_comp_genes = ont_row['genes'].split(' ')
        ont_comp_genes = [x for x in ont_comp_genes if len(x) > 1] #in case theres a weird comma
        n = ont_row['tsize']
        for comp, row in ref_file.iterrows():
            comp_genes = row['genes'].split(' ')
            comp_genes = [x for x in comp_genes if len(x) > 1] #in case theres a weird comma
            N = len(set(comp_genes).intersection(set(hierarchygenes)))
            x = len(set(ont_comp_genes).intersection(set(comp_genes)))
            ref_df.at[ont_comp, comp] = hypergeom.sf(x - 1, M, n, N) # calculare the hypergeometric distribution 
            ji_df.at[ont_comp, comp] = jaccard(set(ont_comp_genes), set(comp_genes)) # calculate the jaccard score
            genecount_df.at[ont_comp, comp] = x
    fdr = multipletests(ref_df.values.flatten(), method='fdr_bh')[1]
    fdr_df = pd.DataFrame(fdr.reshape(ref_df.shape), index=ref_df.index, columns=ref_df.columns, dtype=float)
    for comp, row in fdr_df.iterrows():
        ont_ts.at[comp, 'enriched'] = 0
        ont_ts.at[comp, 'terms'] = ""
        ont_ts.at[comp, 'gene_counts'] = ""
        ont_ts.at[comp, 'adjPvalue'] = ""
        ont_ts.at[comp, 'ji_indexes'] = ""

        tmp = row[row <= fdr_thre]
        
        gene_counts = []
        terms = []
        ji = []
        pvalues = []
        for term in tmp.index:
            if ji_df.at[comp, term] >= ji_thre:
                gene_counts.append(genecount_df.at[comp, term])
                terms.append(term)
                ji.append(ji_df.at[comp,term])
                pvalues.append(tmp[term])
        tmp_df = pd.DataFrame({'gene':gene_counts,'ji': ji, 'terms': terms, 'pvalue': pvalues})
        tmp_df.sort_values(ascending=False, by='ji', inplace=True)
        ont_ts.at[comp, 'enriched'] = len(terms)
        ont_ts.at[comp, 'terms'] = "; ".join(tmp_df['terms'])
        ont_ts.at[comp, 'gene_counts'] =  "; ".join(tmp_df['gene'].astype(str))
        ont_ts.at[comp, 'adjPvalue'] =  "; ".join(tmp_df['pvalue'].astype(str))
        ont_ts.at[comp, 'ji_indexes'] =  "; ".join(tmp_df['ji'].astype(str))

    for comp in ont_ts.index:
        enrich_terms = ji_df.columns[ji_df.loc[comp] == 1].tolist()
        for term in enrich_terms:
            ont_ts.at[comp, 'enriched'] = ont_ts.at[comp, 'enriched'] + 1
            ont_ts.at[comp, 'terms'] = term + "; " + ont_ts.at[comp, 'terms']
            ont_ts.at[comp, 'ji_indexes'] = str(ji_df.at[comp, term]) + "; " + ont_ts.at[comp, 'ji_indexes'] 
            ont_ts.at[comp, 'adjPvalue'] = str(fdr_df.at[comp, term]) + "; " + ont_ts.at[comp, 'adjPvalue']
            ont_ts.at[comp, 'gene_counts'] = str(genecount_df.at[comp, term]) + "; " + ont_ts.at[comp, 'gene_counts']
    print(len(ont_ts))
    print('... finished running geneset enrichment')
    return ont_ts

In [190]:
# Genes in different HPA locations
import json 
with open("/cellar/users/mhu/MuSIC/Resources/u2os_loc.txt", "r") as f:
    geneloc =json.loads(f.read())


In [191]:
loc_df=pd.DataFrame(geneloc.keys(), columns = ['Locations'])
loc_df['genes'] = [x for x in geneloc.values()]
loc_df['tsize'] = loc_df.genes.apply(lambda x: len(x))
loc_df['genes'] = loc_df.genes.apply(lambda x: ' '.join(x))
# loc_df

In [196]:
# hidefdir = '/cellar/users/mhu/MuSIC/U2OS/hidef/MUSE/muse_imgdim_128_ppidim_128_latentd_128_lreg_5_lsuper_10_k_30_euc_True_latent/cutoff_hidef/batch_norm_afterz/'
# hier = 'U2OS_5183_MUSE_batch_norm_afterz_euc_sim_percthre_1.chi_5.maxres_60.alg_louvain'
minTermsize = 4
# f = hidefdir +hier+'.nodes'
f = '/cellar/users/mhu/MuSIC/U2OS/hidef/MUSE/muse_imgdim_128_ppidim_128_latentd_128_lreg_5_lsuper_10_k_30_euc_True_latent/cutoff_hidef/batch_norm_afterz/U2OS_5183_MUSE_batch_norm_afterz_cos_sim_percthre_0.5.chi_15.maxres_20.alg_louvain.nodes'
df = pd.read_table(f, header=None) ## load the input nodes 

df.columns = ['term', 'tsize', 'genes', 'stability']
root_size = df['tsize'].max()

hierarchygenes = df[df['tsize'] == root_size]['genes'].values[0].split(' ')  ## select the root node and collect all genes there (all genes included in the map)
print(f'number of hierarchy genes = {len(hierarchygenes)}')

node_rm = None
if len(df[df['tsize']<minTermsize]) >0:
    node_rm = df[df['tsize']<minTermsize]['term']#the node need to be removed
    print('{} is smaller than minimum term size, Removed'.format(node_rm))  

df = df[(df['tsize'] >= minTermsize) & (df['tsize'] < root_size)]
df.set_index('term', inplace=True)


number of hierarchy genes = 5127


In [197]:
loc_df = loc_df[loc_df['tsize'] >= minTermsize]
loc_df.set_index('Locations', inplace=True)
len(loc_df)

KeyError: "None of ['Locations'] are in the columns"

In [198]:
test = run_hypergeo_enrichr(df, hierarchygenes, loc_df, fdr_thre=0.01, ji_thre = 0, minCompSize=4)
test

100%|██████████| 108/108 [00:01<00:00, 57.86it/s]


108
... finished running geneset enrichment


,tsize,genes,stability,enriched,terms,gene_counts,adjPvalue,ji_indexes
term,,,,,,,,
Cluster1-0,1240,AAGAB AAMDC ABI2 ABL2 ABRAXAS1 ABT1 ACBD6 ACD ...,38,3.0,Nucleoplasm; Nuclear speckles; Nuclear bodies,997.0; 180.0; 115.0,1.3555328319096712e-203; 1.9311729502629996e-7...,0.3984812150279776; 0.14139827179890024; 0.087...
Cluster1-1,821,AAK1 AAR2 ABCE1 ABCF2 ABHD10 ABTB1 ACSF2 ACTR1...,68,1.0,Cytosol,698.0,9.976665413090394e-213,0.3438423645320197
Cluster1-2,753,AAMP ABCA12 ABCB1 ABCB4 ABCC1 ABI1 ACACA ACADM...,28,7.0,Plasma membrane; Actin filaments; Microtubules...,405.0; 84.0; 69.0; 48.0; 47.0; 46.0; 16.0,9.040350741757043e-225; 6.218168378897997e-52;...,0.41838842975206614; 0.10866752910737387; 0.08...
Cluster1-3,659,AACS ABCG1 ABHD17A ABLIM1 ACBD3 ACBD5 ACOX3 AC...,30,8.0,Golgi apparatus; Vesicles; Centrosome; Centrio...,242.0; 286.0; 50.0; 21.0; 14.0; 12.0; 11.0; 10.0,7.278332302669722e-166; 1.4015438351119064e-93...,0.33060109289617484; 0.2643253234750462; 0.071...
Cluster2-0,626,AAMDC ABI2 ABL2 ABT1 ACBD6 ACTL6A ADAM32 ADK A...,57,1.0,Nucleoplasm,608.0,9.425920221317688e-211,0.26701800614844096
...,...,...,...,...,...,...,...,...
Cluster2-43,15,ARFIP1 ARNT BAG1 DNAJB1 DNAJB6 DNAJC13 DPH2 EI...,18,0.0,,,,
Cluster3-45,14,ALDH1A3 ANXA8L1 BCL2L2 CBX5 HES1 IMPA2 LGALSL ...,26,1.0,Nucleoplasm,13.0,0.0037218145216976577,0.005752212389380531
Cluster3-46,11,FNTA GDI1 GDI2 INPP5A MRAS RAP1GDS1 RHOA RRAS ...,29,0.0,,,,


In [76]:
def enrich_eval(df, fdr = 0.01, ji = 0.2):
        term_term_mapping = []
        for index,row in df.iterrows():
            pvalues = str(row[f'adjPvalue']).split('; ')
            jaccard_indexes = str(row[f'ji_indexes']).split('; ')
            hypergeom_terms = str(row[f'terms']).split('; ')
            for i in range(len(pvalues) - 1):
                term_term_mapping.append((index, pvalues[i], hypergeom_terms[i], jaccard_indexes[i]))

        term_term_mapping = pd.DataFrame(term_term_mapping, columns=['term','pval', 'gs_term', 'ji'])
        num_enriched = 0
        num_sig = 0
        taken_gs_terms = []
        taken_terms = []

        for index,row in term_term_mapping.sort_values(by='ji', ascending=False).iterrows():
            if (row['gs_term'] not in taken_gs_terms) & (row['term'] not in taken_terms):
                taken_gs_terms.append(row['gs_term'])
                taken_terms.append(row['term'])
                if float(row['pval']) <= fdr: 
                    num_enriched += 1
                    if (float(row['pval']) <= fdr) & (float(row['ji']) >= ji):
                        num_sig +=1
        return (num_enriched, num_sig)

In [77]:
enrich, sig = enrich_eval(test, fdr = 0.01, ji = 0.2)

In [78]:
print(enrich, sig)

25 17


### Node2Vec embedding hierarchies 

In [17]:
# location of node2vec file
input_dir = "/cellar/users/mhu/MuSIC/U2OS/MuSIC_MUSE/Data"
n2v = load_obj(f'{input_dir}/U2OS_Bioplex/Node2Vec/dim_1024_p_2_q_1.gname_converted.pkl') #load n2v
u2os_genes = np.load('/cellar/users/mhu/MuSIC/U2OS/MuSIC_MUSE/Data/img_x_ppi.5183.genelist.npy', allow_pickle=True)
n2v = n2v[n2v.index.isin(u2os_genes)]  ##filtered for u2os genes

In [18]:
n2v

,1,2,3,4,5,6,7,8,9,10,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024
LPAR1,-0.054839,0.014909,-0.007761,-0.083735,-0.098761,0.025309,0.077098,0.027188,0.012957,-0.013643,...,-0.088257,0.001368,0.028800,0.121568,-0.041672,-0.010725,-0.058908,-0.095545,-0.376645,0.122954
CLGN,0.060697,-0.051199,0.024367,0.042730,0.117842,0.295316,-0.184327,-0.092943,-0.129151,0.004835,...,0.113127,0.075175,-0.052046,0.093476,0.021562,0.022667,-0.001427,-0.200303,-0.008908,-0.080484
VENTX,-0.078559,0.040191,-0.092456,0.050071,-0.024147,-0.158527,-0.075429,0.087791,0.088395,-0.019562,...,-0.172603,0.058839,0.108716,0.066948,-0.037080,0.030158,0.019971,0.179473,0.048359,0.143569
CREB3L2,-0.044460,-0.105897,0.114275,-0.042723,0.107784,0.003660,-0.069222,-0.072611,-0.010452,-0.075588,...,-0.015629,-0.048175,-0.021213,0.082217,0.029391,0.070960,0.007344,0.088688,-0.078115,0.044017
DDX28,-0.060962,0.029248,0.028899,-0.030426,0.045338,-0.183735,0.172176,-0.010493,0.065449,-0.112782,...,-0.012814,-0.064638,-0.037636,-0.019405,-0.018207,0.059696,-0.288469,0.088330,-0.101100,0.181681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WDR12,-0.099054,0.138871,0.029087,-0.064386,-0.005894,0.072527,-0.000873,0.099544,0.018665,-0.050754,...,0.101691,-0.114091,-0.051232,0.176599,0.050801,0.012665,-0.129829,-0.245629,-0.119006,-0.073048
SNX30,-0.008233,-0.065134,0.058258,-0.003312,0.038221,0.048632,0.071484,-0.016234,0.080595,0.095440,...,-0.115892,-0.036842,-0.073982,0.247185,-0.121241,-0.139241,0.031821,-0.018550,-0.206210,0.001165
PSD3,0.030550,-0.064015,0.147363,-0.068088,0.110349,0.039857,-0.037257,-0.087123,0.041750,-0.080156,...,-0.071630,0.014225,-0.006465,0.013971,0.037602,0.020956,-0.082924,0.013400,-0.074433,0.006205
ABCE1,-0.057164,0.100489,-0.002256,0.050893,-0.074641,0.054323,0.062970,0.080352,-0.027381,0.020751,...,0.065886,0.030333,0.042905,0.047653,-0.080272,-0.081742,-0.121114,-0.076036,0.056522,-0.015779


In [19]:
from cellmaps_hierarchyeval.hidef_input_cutoff_utils import *
savedir = '/cellar/users/mhu/MuSIC/U2OS/hidef/Evaluate_Hierarchy/reference_file/Input'
prefix = 'N2V_dim_1024_p_2_q_1.gname_converted'


for dist in ['euc_sim', 'cos_sim']:
    make_sim_matrix(n2v,savedir, prefix, sim_type = dist)
    mat_name = f'{savedir}/{prefix}.{dist}.scaled.pkl'
    # print(mat_name)
    pair = mat_to_pairs(mat_name, sort = False)
    pair_sort = mat_to_pairs(mat_name)

    pair.to_csv('{}/{}_{}.ddot'.format(savedir, prefix, dist), sep='\t', header=False, index=False)
    pair_sort.to_csv('{}/{}_{}_sort.ddot'.format(savedir,prefix, dist), sep='\t', header=False, index=False)



In [27]:
## create cutoff files 

cutoffs = [0.2, 0.5, 1, 2, 5, 10]

label = 'U2OS_5183_N2V_d_1024'
workdir = '/cellar/users/mhu/MuSIC/U2OS/hidef/Input/'
savedir = '/cellar/users/mhu/MuSIC/U2OS/hidef/Evaluate_Hierarchy/reference_file/Input'
prefix = 'N2V_dim_1024_p_2_q_1.gname_converted'

if not os.path.exists(workdir +'N2V/cutoff_hidef'):
    os.makedirs(workdir +'N2V/cutoff_hidef')

for dist_type in ['euc_sim', 'cos_sim']:
    df = pd.read_table('{}/{}_{}_sort.ddot'.format(savedir, prefix, dist_type))
    df.columns =['geneA','geneB', 'weight']
    total_edges = len(df)
    # print(total_edges) #13,429,152
    for cutoff in cutoffs:
        keep = math.ceil((cutoff/100)*total_edges)
            # print(keep) 
        df_cutoff = df.iloc[0:keep]
        df_cutoff.to_csv('{}/N2V/cutoff_hidef/{}_{}_percthre_{}.tsv'.format(workdir,label, dist_type, cutoff),sep='\t', index=False, header=False)

print('Done')    
print(f"Number of files generated: {len(glob('{}/N2V/cutoff_hidef/*.tsv'.format(workdir)))}")
    

Done
Number of files generated: 12


In [52]:
# ## Create the params file to run hidef 

label = 'U2OS_5183_N2V_d_1024'
workdir = '/cellar/users/mhu/MuSIC/U2OS/hidef/Input/N2V'
params = []
stabilities = [5, 10, 15]
resolutions = [20, 40, 60]
algs = ['louvain', 'leiden']
cutoff_files = glob('{}/cutoff_hidef/*.tsv'.format(workdir))

## load the removed list 
with open('/cellar/users/mhu/MuSIC/U2OS/hidef/MUSE/muse_imgdim_128_ppidim_128_latentd_128_lreg_5_lsuper_10_k_30_euc_True_latent/cutoff_hidef/batch_norm_afterz/removed_hierarchy_list.txt', "r") as f:
    removed = f.read().split('\t')
    removed = [s.replace('U2OS_5183_MUSE_batch_norm_afterz', label) for s in removed]

for file in cutoff_files :
    for stability in stabilities:
        for resolution in resolutions:
            for alg in algs: 
                cutoff_fname = file.rsplit('/',1)[-1][:-4]
                g = file
                outname = '{}.chi_{}.maxres_{}.alg_{}'.format(cutoff_fname,stability,resolution, alg)
                if outname in removed:## load do not consider the bad parameter combinations
                    continue
                else:
                    o = '{}/cutoff_hidef/{}'.format(workdir,outname)
                params.append('--g {} --k {} --maxres {} --alg {} --o {}'.format(g,stability,resolution, alg, o))
print(f'number of jobs : {len(params)}')

## this loop creates the params for running layered network hidef
for dist_type in ['euc_sim', 'cos_sim']:
    cutoff_files = glob('{}/cutoff_hidef/*{}*.tsv'.format(workdir, dist_type))
    input_networks = ' '.join(cutoff_files)
    # print(input_networks)
    for stability in stabilities:
        for resolution in resolutions:
            for alg in algs: 
                g = input_networks
                outname = '{}_{}_layered.chi_{}.maxres_{}.alg_{}'.format(label, dist_type,stability,resolution, alg)
                if outname in removed:
                    continue
                else:
                    o = '{}/cutoff_hidef/{}'.format(workdir, outname)
                params.append('--g {} --k {} --maxres {} --alg {} --o {}'.format(g,stability,resolution, alg, o))
print(f'number of jobs : {len(params)}')
                
with open(f'{workdir}/cutoff_hidef/run_cutoff_hidef_weights.param', 'w') as p:
    p.write('\n'.join(params))


if not os.path.exists(f'{workdir}/cutoff_hidef/errfile'):
    os.makedirs(f'{workdir}/cutoff_hidef/errfile')
    
if not os.path.exists(f'{workdir}/cutoff_hidef/outfile'):
    os.makedirs(f'{workdir}/cutoff_hidef/outfile')

number of jobs : 148
number of jobs : 179


### DenseNet embedding hierarchies 

In [60]:
# location of node2vec file
input_dir = "/cellar/users/mhu/MuSIC/U2OS/MuSIC_MUSE/Data"
dn = load_obj(f'{input_dir}/image_data/fold_1/10_fold_embedding.pkl').T #load dn
# dn
u2os_genes = np.load('/cellar/users/mhu/MuSIC/U2OS/MuSIC_MUSE/Data/img_x_ppi.5183.genelist.npy', allow_pickle=True)
dn = dn[dn.index.isin(u2os_genes)]  ##filtered for u2os genes
print(len(dn))

5183


In [62]:
savedir = '/cellar/users/mhu/MuSIC/U2OS/hidef/Evaluate_Hierarchy/reference_file/Input'
prefix = 'DN_d_1024_fold_1'

## create the pair-wise similarity list 
for dist in ['euc_sim', 'cos_sim']:
    #create the similarity matrix
    make_sim_matrix(dn,savedir, prefix, sim_type = dist)
    mat_name = f'{savedir}/{prefix}.{dist}.scaled.pkl'
    # print(mat_name)
    #convert similarity matrix to pairwise similarity table (save the file sort by weight or not)
    pair = mat_to_pairs(mat_name, sort = False)
    pair_sort = mat_to_pairs(mat_name)

    pair.to_csv('{}/{}_{}.ddot'.format(savedir, prefix, dist), sep='\t', header=False, index=False)
    pair_sort.to_csv('{}/{}_{}_sort.ddot'.format(savedir,prefix, dist), sep='\t', header=False, index=False)


In [63]:
## create cutoff files 

cutoffs = [0.2, 0.5, 1, 2, 5, 10]

label = 'U2OS_5183_DN_d_1024_fold_1'
workdir = '/cellar/users/mhu/MuSIC/U2OS/hidef/Input/'
savedir = '/cellar/users/mhu/MuSIC/U2OS/hidef/Evaluate_Hierarchy/reference_file/Input'
prefix = 'DN_d_1024_fold_1'

if not os.path.exists(workdir +'DenseNet/cutoff_hidef'):
    os.makedirs(workdir +'DenseNet/cutoff_hidef')

for dist_type in ['euc_sim', 'cos_sim']:
    df = pd.read_table('{}/{}_{}_sort.ddot'.format(savedir, prefix, dist_type))
    df.columns =['geneA','geneB', 'weight']
    total_edges = len(df)
    # print(total_edges) #13,429,152
    for cutoff in cutoffs:
        keep = math.ceil((cutoff/100)*total_edges)
            # print(keep) 
        df_cutoff = df.iloc[0:keep]
        df_cutoff.to_csv('{}/DenseNet/cutoff_hidef/{}_{}_percthre_{}.tsv'.format(workdir,label, dist_type, cutoff),sep='\t', index=False, header=False)

print('Done')    
print(f"Number of files generated: {len(glob('{}/DenseNet/cutoff_hidef/*.tsv'.format(workdir)))}")
    

Done
Number of files generated: 12


In [65]:
# ## Create the params file to run hidef 

label = 'U2OS_5183_DN_d_1024_fold_1'
workdir = '/cellar/users/mhu/MuSIC/U2OS/hidef/Input/DenseNet'
params = []
stabilities = [5, 10, 15]
resolutions = [20, 40, 60]
algs = ['louvain', 'leiden']
cutoff_files = glob('{}/cutoff_hidef/*.tsv'.format(workdir))
## load the removed list 
with open('/cellar/users/mhu/MuSIC/U2OS/hidef/MUSE/muse_imgdim_128_ppidim_128_latentd_128_lreg_5_lsuper_10_k_30_euc_True_latent/cutoff_hidef/batch_norm_afterz/removed_hierarchy_list.txt', "r") as f:
    removed = f.read().split('\t')
    removed = [s.replace('U2OS_5183_MUSE_batch_norm_afterz', label) for s in removed]

for file in cutoff_files :
    for stability in stabilities:
        for resolution in resolutions:
            for alg in algs: 
                cutoff_fname = file.rsplit('/',1)[-1][:-4]
                g = file
                outname = '{}.chi_{}.maxres_{}.alg_{}'.format(cutoff_fname,stability,resolution, alg)
                if outname in removed:## Do not consider the bad combinations
                    continue
                else:
                    o = '{}/cutoff_hidef/{}'.format(workdir,outname)
                params.append('--g {} --k {} --maxres {} --alg {} --o {}'.format(g,stability,resolution, alg, o))
print(f'number of jobs : {len(params)}')


## this loop creates the params for running layered network hidef
for dist_type in ['euc_sim', 'cos_sim']:
    cutoff_files = glob('{}/cutoff_hidef/*{}*.tsv'.format(workdir, dist_type))
    input_networks = ' '.join(cutoff_files)
    # print(input_networks)
    for stability in stabilities:
        for resolution in resolutions:
            for alg in algs: 
                g = input_networks
                outname = '{}_{}_layered.chi_{}.maxres_{}.alg_{}'.format(label, dist_type,stability,resolution, alg)
                if outname in removed:
                    continue
                else:
                    o = '{}/cutoff_hidef/{}'.format(workdir, outname)
                params.append('--g {} --k {} --maxres {} --alg {} --o {}'.format(g,stability,resolution, alg, o))
print(f'number of jobs : {len(params)}')
                
with open(f'{workdir}/cutoff_hidef/run_cutoff_hidef_weights.param', 'w') as p:
    p.write('\n'.join(params))


if not os.path.exists(f'{workdir}/cutoff_hidef/errfile'):
    os.makedirs(f'{workdir}/cutoff_hidef/errfile')
    
if not os.path.exists(f'{workdir}/cutoff_hidef/outfile'):
    os.makedirs(f'{workdir}/cutoff_hidef/outfile')

number of jobs : 148
number of jobs : 179


# BioPlex Hierarchy 

In [68]:
# location of node2vec file
input_dir = "/cellar/users/mhu/MuSIC/U2OS/MuSIC_MUSE/Data"
bioplex = f'{input_dir}/U2OS_Bioplex/bioplex_edges_x_u2os.txt' #load u2os bioplex

workdir = '/cellar/users/mhu/MuSIC/U2OS/hidef/Input/Bioplex'
if not os.path.exists(workdir +'/cutoff_hidef'):
    os.makedirs(workdir +'/cutoff_hidef')

params = []
stabilities = [5, 10, 15]
resolutions = [20, 40, 60]
algs = ['louvain', 'leiden']

for stability in stabilities:
    for resolution in resolutions:
        for alg in algs: 
            g = bioplex
            outname = 'U2OS_5183_bioplex.chi_{}.maxres_{}.alg_{}'.format(stability,resolution, alg)
            o = '{}/cutoff_hidef/{}'.format(workdir,outname)
            params.append('--g {} --k {} --maxres {} --alg {} --o {}'.format(g,stability,resolution, alg, o))
print(f'number of jobs : {len(params)}')

with open(f'{workdir}/cutoff_hidef/run_cutoff_hidef_weights.param', 'w') as p:
    p.write('\n'.join(params))


if not os.path.exists(f'{workdir}/cutoff_hidef/errfile'):
    os.makedirs(f'{workdir}/cutoff_hidef/errfile')
    
if not os.path.exists(f'{workdir}/cutoff_hidef/outfile'):
    os.makedirs(f'{workdir}/cutoff_hidef/outfile')

number of jobs : 18


In [91]:
workdir = '/cellar/users/mhu/MuSIC/U2OS/hidef/Input/DenseNet/cutoff_hidef/'

n2v_files = glob(f'{workdir}*nodes')
n2v_files[0].rsplit('/',1)[-1].rsplit('_')[2].rsplit('.')[0]
# n2v_files[0].rsplit('/',1)[-1]

'DN'

### Parameters for run enrichr

In [205]:
## Create the param file 
label = 'U2OS_5183_MUSE_batch_norm_afterz'
hidefdir = '/cellar/users/mhu/MuSIC/U2OS/hidef/MUSE/muse_imgdim_128_ppidim_128_latentd_128_lreg_5_lsuper_10_k_30_euc_True_latent/cutoff_hidef/batch_norm_afterz/'
refdir ='/cellar/users/mhu/MuSIC/U2OS/hidef/Input'

hidef_files = glob('{}*nodes'.format(hidefdir))
print(len(hidef_files))
with open('{}removed_hierarchy_list.txt'.format(hidefdir), "r") as f:
    removed = f.read().split('\t')
    removed = [hidefdir+i+'.nodes' for i in removed]
print(len(removed))
hidef_analyze= set(hidef_files) - set(removed)
print(len(hidef_analyze))

## obtain all the nodes files from N2V, DN and bioplex
N2V_files = glob(f'{refdir}/N2V/cutoff_hidef/*nodes')
DN_files = glob(f'{refdir}/DenseNet/cutoff_hidef/*nodes')
bioplex_files = glob(f'{refdir}/Bioplex/cutoff_hidef/*nodes')
print(len(N2V_files), len(DN_files), len(bioplex_files))

params = []
for hidef_file in hidef_analyze:
    reflist = []
    hier = hidef_file.rsplit('/',1)[-1]
    # print(hier)
    param = hier.replace('U2OS_5183_MUSE_batch_norm_afterz_','') # remove the prefix label 
    reflist += [x for x in DN_files if param in x]+[x for x in N2V_files if param in x] # find the DN and N2V hierarchy with the same param settings 
    hidef_param = '.'.join(param.rsplit('.')[-4:]) # only keep the part with chi, maxres and alg 
    # print(hidef_param)
    reflist += [x for x in bioplex_files if hidef_param in x]
    assert (len(reflist) == 3), f'missing refs for {hier}'
    # print(len(reflist))
    reflist = ' '.join(reflist) # when pass into args, it does not accept comma
    
    prefix = hidef_file[:-6]
    
    params.append('--outprefix {} --infname {}  --refhier {}'.format(prefix, hidef_file, reflist))
    
print(f'number of jobs : {len(params)}')

with open(f'/cellar/users/mhu/MuSIC/U2OS/hidef//run_enrichr_input.param', 'w') as p:
    p.write('\n'.join(params))


252
73
179
179 179 18
number of jobs : 179


#### Test run
Confirm the program is working 

In [203]:
python = '/cellar/users/mhu/miniconda3/envs/hidef/bin/python '
test_run = f'/cellar/users/mhu/MuSIC/U2OS/hidef/Evaluate_Hierarchy/Input_hierarchy_enrichment_in_MuSIC.py {params[0]}'
# python + test_run
os.system(python + test_run)

number of hierarchy genes = 5127
Number of hierarchy nodes: 108
type of ref is DN
number of hierarchy genes = 4644
Number of hierarchy nodes: 74


  1%|          | 1/108 [00:00<00:14,  7.47it/s]

Number of nodes: 108
... finished running geneset enrichment
type of ref is N2V
number of hierarchy genes = 4871
Number of hierarchy nodes: 149


  2%|▏         | 2/108 [00:00<00:08, 11.92it/s]

Number of nodes: 108
... finished running geneset enrichment
type of ref is bioplex
number of hierarchy genes = 4757
Number of hierarchy nodes: 114


...

Number of nodes: 108
... finished running geneset enrichment
... finished running enrichment analysis
4704
... finished loading PPI data


 99%|█████████▉| 107/108 [03:09<00:01,  1.47s/it]

... finished running edge enrichment analysis
=== finished analyze_hidef_output ===


100%|██████████| 108/108 [03:11<00:00,  1.77s/it]


0